## Import Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from fastai.text.all import *
%matplotlib inline
pd.options.display.max_columns = None

## Import Data

In [2]:
dataPath = Path("../data/shopify/")
[file for file in dataPath.glob('*')]

[Path('../data/shopify/models'),
 Path('../data/shopify/raw'),
 Path('../data/shopify/shopify_apps.csv')]

In [3]:
apps_data = pd.read_csv(dataPath/"shopify_apps.csv")
print(apps_data.shape)
apps_data.head()

(4750, 6)


,app_id,url,title,description,tagline,category
0,9e4748a9-7eda-4814-83b6-0537d44152b1,https://apps.shopify.com/translate,Panda Language Translate,Panda Language Translation improve your business Internationally \n\n Every store owners would like to: \n\n Increase sales by acquiring new customers\nImprove conversion by speaking your visitors language \n\n Many of customers might leave your store because they are not: \n\n \n Redirected to your shop matching their preferred language \n Allow an option to choose their own preferred language with Flag option \n Customers from all around the world expect your store to be available in their preferred language \n \n\n Translate the gap by allowing easily provide multilingual translate tool...,Translate your store into multiple languages,Store design
1,d1476138-a608-4bb9-8d39-b30f3ca7617d,https://apps.shopify.com/instant-brand-page,Instant Brand Page,Having a brand page gives your shoppers a quick way to see the variety in your inventory and identify with their favourite brands. \n No coding required and image support without needing a collection per brand!! Key Features \n \n Three different layouts to choose from. \n Style your brand page how you want it. \n Automatic matching to collections - saves you time! \n Use collection images or upload your own custom brand image. \n Use sale icons to show your customers their favourite brands with discounted products. \n Sale icons will display dynamically if set - no management required! \n...,A-Z Brand Index Page and Favourites Slider,Store design
2,d6e49a3c-2f9f-4bfa-8c26-5d024faf2241,https://apps.shopify.com/powr-faq,FAQ Accordion | Help Center,"Create, organize and display Frequently Asked Questions on any page. \n Make finding an answer to common questions easy! With POWr FAQ Accordion you can: \n\n \n Reduce support requests by adding answers to frequent questions about product info to your website. \n Include shipping details, your refund policy and more on your Help Centre page. \n Use an accordian layout to make it easy to browse through different questions. \n Expand answers to your most frequent questions. \n Include text, links, images, and videos in your answers. \n Add a search bar to make your FAQ accordion searchable....","FAQ page, FAQ accordion menu for product Info & refund policy","Store design,Customer support"
3,0ef0087f-3ae5-4dbc-84e0-193b576d82ed,https://apps.shopify.com/mps-promote-me,Promote Me | Many apps in one,"Promote Me app is a bundle of apps that includes many Shopify apps, aimed at boosting sales by increasing conversion rate and decreasing page load time. We will including more and more apps as app updates. \n Finetune Store Apps \n \n Back to Top button \n Open External Links in New Tabs \n Terms and Conditions on Cart \n Skip Cart on Product page \n Remove Shopify Branding \n Geolocation Redirect \n \n Protect Store Apps \n \n Advanced Cookie Bar \n Copy Protection Plus \n Hide Best Selling Filter \n \n Improve Experience Apps \n \n Search Autocomplete \n Instant Page Loader \n Currency C...","Spin Wheel,Currency Converter,Quick ATC Buttons & 18+ apps...","Store design,Sales and conversion optimization"
4,7aac2a1f-ff03-4f38-aeb7-7619403a6f05,https://apps.shopify.com/installify-boost-mobile-app-installs,Instalify,We are trusted by hundreds of Shopify and Shopify Plus customers from around the world. \n Let your customers know that you have now launched your mobile apps. Guide them to the App Store or Play Store to easily download and start using your app. We have seen our customers who use Instalify get more installs saving them thousands of dollars in marketing. \n 3 Key Benefits: \n \n Increase Mobile App Installs- Organically increase your mobile app installs by guiding your website customers to your app. \n Easy Setup- Quick and simple set up that takes less than 5 minutes \n Powerful Analytics...,Supercharge Your Mobile App Installs,Store design


## Training a language Model

In [4]:
## Creating a Text Data loader for language model
dls_lm = TextDataLoaders.from_df(apps_data, text_col=3, is_lm=True)
dls_lm.show_batch(max_n=5)

,text,text_
0,xxbos xxmaj automatically sync your xxmaj shopify orders to your quickbooks xxmaj online account and enjoy seamless financial reporting with just a couple of clicks . \n▁ xxmaj automatic xxmaj daily xxmaj sync \n▁ xxmaj quickly and easily import all your orders and refunds from xxmaj shopify into quickbooks xxmaj online . xxmaj you can choose to automatically sync order data and export historical data from your xxmaj shopify account . \n▁,xxmaj automatically sync your xxmaj shopify orders to your quickbooks xxmaj online account and enjoy seamless financial reporting with just a couple of clicks . \n▁ xxmaj automatic xxmaj daily xxmaj sync \n▁ xxmaj quickly and easily import all your orders and refunds from xxmaj shopify into quickbooks xxmaj online . xxmaj you can choose to automatically sync order data and export historical data from your xxmaj shopify account . \n▁ xxmaj
1,"social proof application that shows xxmaj live visitor count , xxmaj recent order notifications , and xxmaj overall total sales as well . \n▁ xxmaj but what is social proof ? \n▁ xxmaj if you ’ve ever tried to book a hotel online , you ’d have seen small popups suggesting ' how many people are booking a room ' , ' who has recently booked that hotel ' , and '","proof application that shows xxmaj live visitor count , xxmaj recent order notifications , and xxmaj overall total sales as well . \n▁ xxmaj but what is social proof ? \n▁ xxmaj if you ’ve ever tried to book a hotel online , you ’d have seen small popups suggesting ' how many people are booking a room ' , ' who has recently booked that hotel ' , and ' how"
2,"easy and xxmaj affordable . xxmaj as your smart virtual assistant available 24 / 7 , xxmaj mark will be able to analyze your store and your advertising campaigns to help you make strategic marketing decisions . \n▁ xxmaj mark can also analyze all your metrics from different data sources like xxmaj google xxmaj analytics and give you a benchmark on all your data . \n▁ xxmaj here is what xxmaj mark","and xxmaj affordable . xxmaj as your smart virtual assistant available 24 / 7 , xxmaj mark will be able to analyze your store and your advertising campaigns to help you make strategic marketing decisions . \n▁ xxmaj mark can also analyze all your metrics from different data sources like xxmaj google xxmaj analytics and give you a benchmark on all your data . \n▁ xxmaj here is what xxmaj mark can"
3,"a xxmaj stories format and get to see more options with no or minimal effort , ending up spending more time on your website . \n▁ xxmaj stories xxmaj analytics : \n▁ xxmaj get to know how xxmaj stories perform for you and analytics on views , engagements , and clicks . \n▁ xxmaj how does it work ? \n▁ xxmaj step 1 : xxmaj upload an image : you can reuse","xxmaj stories format and get to see more options with no or minimal effort , ending up spending more time on your website . \n▁ xxmaj stories xxmaj analytics : \n▁ xxmaj get to know how xxmaj stories perform for you and analytics on views , engagements , and clicks . \n▁ xxmaj how does it work ? \n▁ xxmaj step 1 : xxmaj upload an image : you can reuse the"
4,"free forever plan : xxmaj use the rewards program without any limits \n▁ xxmaj easy to use : xxmaj simply install and the program will be setup instantly \n▁ xxmaj fully customizable : xxmaj customize the colours , choose your font , have full xxup html control over the emails to fit 100 % to your branding \n▁\n▁ 3 xxmaj rewards program you can start utilizing \n▁\n▁ xxmaj points xxmaj rewards xxmaj","forever plan : xxmaj use the rewards program without any limits \n▁ xxmaj easy to use : xxmaj simply install and the program will be setup instantly \n▁ xxmaj fully customizable : xxmaj customize the colours , choose your font , have full xxup html control over the emails to fit 100 % to your branding \n▁\n▁ 3 xxmaj rewards program you can start utilizing \n▁\n▁ xxmaj points xxmaj rewards xxmaj program"


In [5]:
## Initiating the language model
learn = language_model_learner(
    dls_lm, 
    AWD_LSTM, 
    metrics=[accuracy, Perplexity()], 
    path=dataPath, 
    wd=0.1).to_fp16()

In [6]:
## Training last layer
learn.fit_one_cycle(5, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.401289,3.894527,0.301209,49.132816,01:06
1,4.082547,3.718361,0.316634,41.196812,01:05
2,3.962107,3.647734,0.324348,38.387589,01:05
3,3.848866,3.604662,0.329019,36.769241,01:05
4,3.773146,3.595030,0.330209,36.416805,01:05


In [7]:
## Training all layers
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.647418,3.511912,0.342721,33.512295,01:19
1,3.519794,3.408947,0.357038,30.233391,01:19
2,3.395638,3.349552,0.365107,28.489979,01:19
3,3.303498,3.317507,0.370716,27.591473,01:19
4,3.270912,3.310615,0.371249,27.401974,01:19


In [8]:
## Saving encoder 
learn.save_encoder('finetuned')

## Training a classifier

In [9]:
apps_data.head()

,app_id,url,title,description,tagline,category
0,9e4748a9-7eda-4814-83b6-0537d44152b1,https://apps.shopify.com/translate,Panda Language Translate,Panda Language Translation improve your business Internationally \n\n Every store owners would like to: \n\n Increase sales by acquiring new customers\nImprove conversion by speaking your visitors language \n\n Many of customers might leave your store because they are not: \n\n \n Redirected to your shop matching their preferred language \n Allow an option to choose their own preferred language with Flag option \n Customers from all around the world expect your store to be available in their preferred language \n \n\n Translate the gap by allowing easily provide multilingual translate tool...,Translate your store into multiple languages,Store design
1,d1476138-a608-4bb9-8d39-b30f3ca7617d,https://apps.shopify.com/instant-brand-page,Instant Brand Page,Having a brand page gives your shoppers a quick way to see the variety in your inventory and identify with their favourite brands. \n No coding required and image support without needing a collection per brand!! Key Features \n \n Three different layouts to choose from. \n Style your brand page how you want it. \n Automatic matching to collections - saves you time! \n Use collection images or upload your own custom brand image. \n Use sale icons to show your customers their favourite brands with discounted products. \n Sale icons will display dynamically if set - no management required! \n...,A-Z Brand Index Page and Favourites Slider,Store design
2,d6e49a3c-2f9f-4bfa-8c26-5d024faf2241,https://apps.shopify.com/powr-faq,FAQ Accordion | Help Center,"Create, organize and display Frequently Asked Questions on any page. \n Make finding an answer to common questions easy! With POWr FAQ Accordion you can: \n\n \n Reduce support requests by adding answers to frequent questions about product info to your website. \n Include shipping details, your refund policy and more on your Help Centre page. \n Use an accordian layout to make it easy to browse through different questions. \n Expand answers to your most frequent questions. \n Include text, links, images, and videos in your answers. \n Add a search bar to make your FAQ accordion searchable....","FAQ page, FAQ accordion menu for product Info & refund policy","Store design,Customer support"
3,0ef0087f-3ae5-4dbc-84e0-193b576d82ed,https://apps.shopify.com/mps-promote-me,Promote Me | Many apps in one,"Promote Me app is a bundle of apps that includes many Shopify apps, aimed at boosting sales by increasing conversion rate and decreasing page load time. We will including more and more apps as app updates. \n Finetune Store Apps \n \n Back to Top button \n Open External Links in New Tabs \n Terms and Conditions on Cart \n Skip Cart on Product page \n Remove Shopify Branding \n Geolocation Redirect \n \n Protect Store Apps \n \n Advanced Cookie Bar \n Copy Protection Plus \n Hide Best Selling Filter \n \n Improve Experience Apps \n \n Search Autocomplete \n Instant Page Loader \n Currency C...","Spin Wheel,Currency Converter,Quick ATC Buttons & 18+ apps...","Store design,Sales and conversion optimization"
4,7aac2a1f-ff03-4f38-aeb7-7619403a6f05,https://apps.shopify.com/installify-boost-mobile-app-installs,Instalify,We are trusted by hundreds of Shopify and Shopify Plus customers from around the world. \n Let your customers know that you have now launched your mobile apps. Guide them to the App Store or Play Store to easily download and start using your app. We have seen our customers who use Instalify get more installs saving them thousands of dollars in marketing. \n 3 Key Benefits: \n \n Increase Mobile App Installs- Organically increase your mobile app installs by guiding your website customers to your app. \n Easy Setup- Quick and simple set up that takes less than 5 minutes \n Powerful Analytics...,Supercharge Your Mobile App Installs,Store design


In [18]:
apps_data.columns

Index(['app_id', 'url', 'title', 'description', 'tagline', 'category'], dtype='object')

In [22]:
dls_blk  = DataBlock(
    blocks = (TextBlock.from_df(text_cols="description", seq_len=128),
            MultiCategoryBlock),
    get_x = ColReader(cols="text"),
    get_y = ColReader(cols="category", label_delim=","),
    splitter = TrainTestSplitter(test_size=0.2, random_state=21)
)

dls_clf = dls_blk.dataloaders(apps_data, bs =16, seed =20)

In [23]:
## Checking if everything is working fine
dls_clf.show_batch(max_n=5)

,text,None
0,"xxbos shippypro is the one - stop xxmaj shipping xxmaj solution that allows you to create xxmaj shipping xxmaj labels , xxmaj track xxmaj packages and xxmaj manage xxmaj returns . xxmaj give it a try , the first 30 orders are xxup free . \n▁ xxmaj integrates with 110 + carriers including : \n▁ fedex , xxup ups , xxup dhl xxmaj express , xxup xxunk , xxmaj poste xxmaj xxunk , xxup gls xxmaj italy , xxmaj xxunk , xxup tnt xxup it , xxup xxunk , xxmaj asendia , xxunk , xxup b2c xxmaj europe , xxup xxunk , xxunk , xxmaj xxunk , xxup dhl xxmaj parcel , xxup xxunk , xxup dpd xxup hr , xxup gls xxup ee , xxmaj correos , xxmaj xxunk , xxup gls xxup es , xxmaj xxunk , xxmaj xxunk , xxmaj correos xxmaj express , xxup mrw ,",Orders and shipping
1,"xxbos xxup faq xxmaj page & xxmaj accordion app by xxup ndnapps enables customers to look for common inquiries , questions with answers on the xxmaj knowledge xxmaj base and xxup faq lists . xxmaj admins can create their own informative seo - friendly xxup faq page with all necessary features . \n▁ xxmaj the features of xxup faq xxmaj page & xxmaj accordion app by xxup ndnapps \n▁\n▁ xxmaj work effectively on all devices ( desktop , xxmaj tablet , xxmaj mobile ) : xxmaj you can create a nice responsive xxup faq page ( help xxmaj center ) easily without technical knowledge . xxmaj this app supports xxunk / expanding features to viewing questions . \n▁ xxmaj making rich results in xxmaj google search ( good for xxup seo ) . \n▁ xxmaj use an embed code to show xxup faq on any front - end pages .",Customer support;Store design
2,"xxbos xxmaj click "" add xxmaj app "" for a free trial of the xxup seo tools that over 30 , xxrep 3 0 stores and 100 ’s of xxmaj shopify xxmaj plus stores use to optimize xxup seo and drive traffic . \n\n▁ xxmaj cancel at any time and keep your xxunk start optimizing now . \n▁ xxmaj the xxmaj complete xxup seo xxmaj app \n▁ xxmaj check , fix and optimize with the xxup seo all in one power tool : xxmaj plug in xxup seo . xxmaj super easy to set up and use , it will help you with everything you need to improve your xxmaj google ranking , drive organic traffic to your shop and manage xxup seo optimization . \n\n▁\n▁ xxmaj structured xxmaj data ( json - ld xxmaj schema xxmaj markup ) aka . xxmaj rich xxmaj snippets \n▁ xxmaj bulk xxmaj edit",Marketing
3,"xxbos xxmaj flexify syncs xxmaj your xxmaj products with xxmaj facebook xxmaj product xxmaj catalog & allows you to create xxmaj dynamic xxmaj product xxmaj ads . xxmaj put your product ads in front of millions of people on xxmaj facebook , xxmaj instagram & xxmaj google without manual work . xxmaj these highly targeted ads follow prospects across all their devices & are always up - to - date thanks to xxmaj flexify 's social product feed . xxmaj true set & forget advertising ! \n▁ xxmaj join the thousands of merchants who have chosen xxmaj flexify . xxmaj get started xxup today risk - free with our xxup free - forever plan ! xxmaj setup is easy . xxmaj we offer xxup real xxup support & keep this app xxup up - to - date . \n▁ xxmaj how we help you succeed \n▁ xxmaj social xxmaj commerce",Marketing
4,"xxbos xxup new - xxmaj automatic xxmaj unique xxmaj coupons - xxmaj provide each subscriber with a completely unique coupon code that can only be used once and ca n't be reused . \n\n▁ xxup new - a / b xxmaj test - xxmaj create two games or more , run them simultaneously , and analyze each one ’s conversion rates . xxmaj based on these statistics , you will easily find the winning combination of texts , colors , and images . \n▁ woohoo xxmaj popup xxmaj games : xxup spin xxup the xxup wheel xxup pick a xxup gift xxup card xxup dance xxup xxunk xxup of xxup coupons \n▁ xxmaj increase your subscriber 's list by displaying exit - intent coupon popups . \n▁ xxup exit xxup intent xxup trigger xxup included ! xxmaj key features \n▁\n▁ xxmaj smart xxmaj triggers \n▁ xxmaj custom xxmaj images \n▁",Marketing;Sales and convers

In [24]:
## Defining our text classifier
learn = text_classifier_learner(dls_clf , AWD_LSTM, drop_mult=1, metrics=accuracy_multi)
learn.path = dataPath

In [25]:
## Loading Language model encoder weights trained in previous section
learn = learn.load_encoder(file='finetuned')

In [26]:
## Finetuning last layer
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy_multi,time
0,0.387388,0.194760,0.918158,01:10


In [27]:
## Finetuning last 2 layers and progressive learning rate
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy_multi,time
0,0.290416,0.189930,0.920790,01:17


In [28]:
## Finetuning last 3 layers and progressive learning rate
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy_multi,time
0,0.257554,0.186051,0.922895,01:40


In [29]:
## Training entire model
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy_multi,time
0,0.231401,0.183183,0.924211,02:00
1,0.221636,0.178899,0.926579,02:00
2,0.214045,0.177762,0.925789,02:00
3,0.200994,0.175965,0.927895,02:00
4,0.198360,0.176549,0.927544,02:00


In [30]:
## Save model weights
learn.save('classifier')

Path('../data/shopify/models/classifier.pth')